In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('transactions.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df.head()

,created_at,customer_id,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount
0,2018-07-29 15:22:01.458193+00:00,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,"[{'product_id': 54728, 'quantity': 1, 'item_pr...",Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832
1,2018-07-30 12:40:22.365620+00:00,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,"[{'product_id': 16193, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526
2,2018-09-15 11:51:17.365620+00:00,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,"[{'product_id': 53686, 'quantity': 4, 'item_pr...",OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696
3,2018-11-01 11:23:48.365620+00:00,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,"[{'product_id': 20228, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012
4,2018-12-18 11:20:30.365620+00:00,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,"[{'product_id': 55220, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753


In [3]:
df['created_at'].min()

Timestamp('2016-06-30 23:18:44.792905+0000', tz='UTC')

In [4]:
df['created_at'].max()

Timestamp('2022-07-31 23:59:45.821469+0000', tz='UTC')

### Make sure to filters only transactions which were created during the last year.

In [5]:
df = df[(df['created_at'] >= '2021-08-01') & (df['created_at'] <= '2022-07-31')]

In [6]:
df['created_at'].min()


Timestamp('2021-08-01 00:01:46.133313+0000', tz='UTC')

### Drop the unnecessary columns which won't be needed for the analysis.  

In [7]:
df = df.drop(columns=['promo_amount', 'shipment_location_lat','shipment_location_long', 'shipment_fee', 'promo_code',"shipment_date_limit" ])

In [8]:
df.head()

,created_at,customer_id,booking_id,session_id,product_metadata,payment_method,payment_status,total_amount
25,2021-08-31 11:05:02.365620+00:00,4774,91d7b8a0-f3f5-4a0c-843a-f83a75b415e2,b2fb0889-6e08-4bc5-b6ba-2ac71d1b3145,"[{'product_id': 10460, 'quantity': 4, 'item_pr...",Credit Card,Success,2091956
26,2021-10-17 11:45:13.365620+00:00,4774,6a32b1ef-142a-41f8-8046-f9b74676233d,ced773bd-a755-4e23-a1ab-4a96ef9c75e1,"[{'product_id': 2334, 'quantity': 1, 'item_pri...",Credit Card,Success,450657
27,2021-12-03 11:55:49.365620+00:00,4774,e319fd63-6af2-4103-bf76-4b84658ac92f,e3bc7255-d3ad-4be9-98f4-f60efcc6f840,"[{'product_id': 18477, 'quantity': 1, 'item_pr...",Credit Card,Success,488031
28,2022-01-19 11:25:46.365620+00:00,4774,0d1342a0-bac8-4837-bea6-4196033422a5,5d3d5de2-3a5a-42e9-8f8c-512346a4c031,"[{'product_id': 33212, 'quantity': 3, 'item_pr...",Credit Card,Success,1078709
29,2022-03-07 10:53:30.365620+00:00,4774,e9597c94-dd17-4a93-81d9-aae586182d87,514d9158-5020-4dd9-8ca0-22097c600b78,"[{'product_id': 21995, 'quantity': 1, 'item_pr...",Debit Card,Success,311231


### I can see the the column product_metada is quite messy, let's take a closer look how it's structured.

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df['product_metadata'].head(10)

25     [{'product_id': 10460, 'quantity': 4, 'item_price': 398497}, {'product_id': 31208, 'quantity': 1, 'item_price': 487968}]
26                                                                  [{'product_id': 2334, 'quantity': 1, 'item_price': 443409}]
27                                                                 [{'product_id': 18477, 'quantity': 1, 'item_price': 481861}]
28                                                                 [{'product_id': 33212, 'quantity': 3, 'item_price': 357903}]
29                                                                 [{'product_id': 21995, 'quantity': 1, 'item_price': 308486}]
30                                                                 [{'product_id': 51055, 'quantity': 1, 'item_price': 114212}]
31                                                                 [{'product_id': 18476, 'quantity': 1, 'item_price': 198544}]
106                                                                [{'product_id': 33692, 'quantity': 8,

### I split the column into 3 new different columns ProductID, Quantity and Item_Price since it will be easier to read.

In [10]:
import ast

df["product_metadata"] = df["product_metadata"].apply(ast.literal_eval)

df = df.explode("product_metadata")

df["product_id"] = df["product_metadata"].apply(lambda x: x["product_id"])
df["quantity"] = df["product_metadata"].apply(lambda x: x["quantity"])
df["item_price"] = df["product_metadata"].apply(lambda x: x.get("item_price", 0))

df = df.drop(columns=["product_metadata"])

In [11]:
df.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,total_amount,product_id,quantity,item_price
25,2021-08-31 11:05:02.365620+00:00,4774,91d7b8a0-f3f5-4a0c-843a-f83a75b415e2,b2fb0889-6e08-4bc5-b6ba-2ac71d1b3145,Credit Card,Success,2091956,10460,4,398497
25,2021-08-31 11:05:02.365620+00:00,4774,91d7b8a0-f3f5-4a0c-843a-f83a75b415e2,b2fb0889-6e08-4bc5-b6ba-2ac71d1b3145,Credit Card,Success,2091956,31208,1,487968
26,2021-10-17 11:45:13.365620+00:00,4774,6a32b1ef-142a-41f8-8046-f9b74676233d,ced773bd-a755-4e23-a1ab-4a96ef9c75e1,Credit Card,Success,450657,2334,1,443409
27,2021-12-03 11:55:49.365620+00:00,4774,e319fd63-6af2-4103-bf76-4b84658ac92f,e3bc7255-d3ad-4be9-98f4-f60efcc6f840,Credit Card,Success,488031,18477,1,481861
28,2022-01-19 11:25:46.365620+00:00,4774,0d1342a0-bac8-4837-bea6-4196033422a5,5d3d5de2-3a5a-42e9-8f8c-512346a4c031,Credit Card,Success,1078709,33212,3,357903


### Since all the prices are in Indonesian Rupiah, I will convert them to USD so they are more easily interpreted.
### 1 USD = 16,555 IDR

In [12]:
exchange_rate = 16555  

df['item_price'] = (df['item_price'] / exchange_rate).round(2)
df['total_amount'] = (df['total_amount'] / exchange_rate).round(2)

In [14]:
df.head(15)

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,total_amount,product_id,quantity,item_price
25,2021-08-31 11:05:02.365620+00:00,4774,91d7b8a0-f3f5-4a0c-843a-f83a75b415e2,b2fb0889-6e08-4bc5-b6ba-2ac71d1b3145,Credit Card,Success,126.36,10460,4,24.07
25,2021-08-31 11:05:02.365620+00:00,4774,91d7b8a0-f3f5-4a0c-843a-f83a75b415e2,b2fb0889-6e08-4bc5-b6ba-2ac71d1b3145,Credit Card,Success,126.36,31208,1,29.48
26,2021-10-17 11:45:13.365620+00:00,4774,6a32b1ef-142a-41f8-8046-f9b74676233d,ced773bd-a755-4e23-a1ab-4a96ef9c75e1,Credit Card,Success,27.22,2334,1,26.78
27,2021-12-03 11:55:49.365620+00:00,4774,e319fd63-6af2-4103-bf76-4b84658ac92f,e3bc7255-d3ad-4be9-98f4-f60efcc6f840,Credit Card,Success,29.48,18477,1,29.11
28,2022-01-19 11:25:46.365620+00:00,4774,0d1342a0-bac8-4837-bea6-4196033422a5,5d3d5de2-3a5a-42e9-8f8c-512346a4c031,Credit Card,Success,65.16,33212,3,21.62
29,2022-03-07 10:53:30.365620+00:00,4774,e9597c94-dd17-4a93-81d9-aae586182d87,514d9158-5020-4dd9-8ca0-22097c600b78,Debit Card,Success,18.80,21995,1,18.63
30,2022-04-23 11:43:52.365620+00:00,4774,e82a1b9f-7a30-431b-af7d-8108f6aca0c9,7e644152-6798-4511-8c27-0648ebff6e17,Credit Card,Success,7.08,51055,1,6.90
31,2022-06-09 12:18:09.365620+00:00,4774,7ab6eac3-ff92-46d7-9030-296301ee920a,1688279e-d03f-466a-a20e-57846de4c179,Credit Card,Success,14.81,18476,1,11.99
106,2021-08-12 18:26:59.162570+00:00,58191,0f9cd115-add4-4dbd-9707-fc50233cc535,693a57c0-7102-4612-bc16-2b5726be9513,OVO,Success,78.67,33692,8,9.64
107,2021-08-27 19:14:52.162570+00:00,58191,42730372-cf0d-46fc-8741-7c788927fd61,f97334ad-f046-422a-9c69-a4b25c99a087,OVO,Success,26.19,45375,1,25.58


In [16]:
df.dtypes

created_at        datetime64[ns, UTC]
customer_id                     int64
booking_id                     object
session_id                     object
payment_method                 object
payment_status                 object
total_amount                  float64
product_id                      int64
quantity                        int64
item_price                    float64
dtype: object

### Convert some of the columns to more ideal datatypes.

In [17]:
df = df.astype({
    'customer_id': 'int32',
    'payment_method': 'category',
    'payment_status': 'category',
    'product_id': 'int32',
    'quantity': 'int16',
    'total_amount': 'float32'
})

print(df.dtypes)

created_at        datetime64[ns, UTC]
customer_id                     int32
booking_id                     object
session_id                     object
payment_method               category
payment_status               category
total_amount                  float32
product_id                      int32
quantity                        int16
item_price                    float64
dtype: object


In [18]:
df.describe()

,customer_id,total_amount,product_id,quantity,item_price
count,462548.00000,462548.000000,462548.000000,462548.000000,462548.000000
mean,49873.38452,71.147568,29687.917998,1.472561,15.118145
std,29124.62280,104.048698,17063.022060,1.580071,6.758329
min,3.00000,0.680000,1163.000000,1.000000,0.680000
25%,24407.00000,14.900000,14745.000000,1.000000,10.180000
50%,49767.50000,27.450001,28592.500000,1.000000,14.120000
75%,75243.75000,81.930000,44692.000000,1.000000,18.980000
max,99998.00000,1311.010010,60000.000000,42.000000,70.080000


In [19]:
df.to_csv('transactions_cleaned.csv', index=False)